In [1]:
import os
import re
import logging
import numpy as np

import tflearn
from tflearn.data_utils import image_preloader
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_3d, max_pool_3d, conv_3d_transpose
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
import tensorflow as tf
from tensorflow.contrib.keras.api.keras import backend as K
import math
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import FunctionTransformer
from matplotlib.figure import figaspect
from scipy.integrate import trapz

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
BATCH_SIZE = 5000
LR = 0.001

In [3]:
def build_model(n_1, n_2, n_3):   # build computational graph
    network = input_data(shape=(None, n_1*n_2*n_3), name = 'input')
    network = fully_connected(network, 1000, activation = 'relu')
    network = fully_connected(network, 500, activation = 'relu')
    network = fully_connected(network, 1, activation = 'linear')

    return regression(network, optimizer='adam',
                      loss='mean_square', learning_rate=LR, name = 'targets')

    
def predict_gga(inp, rotate_type = 0): # input = 3D density array, output = 3D Vxc potential
    a = inp.shape[0]
    b = inp.shape[1]
    c = inp.shape[2]
    result_prediction = np.zeros((a,b,c))
    for i in range(a):
        for k in range(b):
            for l in range(c):
                indices1 = range(i-2,i+3)
                indices2 = range(k-2,k+3)
                indices3 = range(l-2,l+3)
                neighbourhood = inp.take(indices1, axis = 0, mode='wrap')
                neighbourhood = neighbourhood.take(indices2, axis = 1, mode='wrap')
                neighbourhood = neighbourhood.take(indices3, axis = 2, mode='wrap')
                if rotate_type == 0:
                    neighbourhood_final = neighbourhood
                if rotate_type == 1:
                    neighbourhood_final = np.rot90(neighbourhood, 1, (0,1))
                if rotate_type == 2:
                    neighbourhood_final = np.rot90(neighbourhood, 1, (1,2))
                result_prediction[i,k,l] = model_gga.predict(np.reshape(neighbourhood_final, (1, -1)))
    return result_prediction

def predict_lda(inp): # input = 3D density array, output = 3D Vxc potential
    a = inp.shape[0]
    b = inp.shape[1]
    c = inp.shape[2]
    result_prediction = np.zeros((a,b,c))
    for i in range(a):
        for k in range(b):
            for l in range(c):
                result_prediction[i,k,l] = model_lda.predict(np.reshape(inp[i,k,l], (-1, 1)))
    return result_prediction

In [7]:
tf.reset_default_graph()

In [5]:
# Build LDA NN
lda_network = build_model(1,1,1)
model_lda = tflearn.DNN(lda_network, tensorboard_verbose=0)

In [6]:
# Load LDA model from file
model_lda.load('lda_model')

INFO:tensorflow:Restoring parameters from C:\Users\Atty\Desktop\Quantum ML\MLXC_ver4\ML_XC-master\octopus_NN\lda_model


In [8]:
# Build GGA NN
gga_network = build_model(5,5,5)
model_gga = tflearn.DNN(gga_network, tensorboard_verbose=0)

In [9]:
# load GGA model from file
model_gga.load('gga_model')

INFO:tensorflow:Restoring parameters from C:\Users\Atty\Desktop\Quantum ML\MLXC_ver4\ML_XC-master\octopus_NN\gga_model
